In [3]:
import pandas as pd
from shapely.geometry import shape
import json

# Load the GeoJSON file
file_path = '../data/HDBExistingBuilding.geojson'
with open(file_path, 'r') as f:
    geojson_data = json.load(f)
centroids_data = []

for feature in geojson_data['features']:
    polygon = shape(feature['geometry'])
    centroid = polygon.centroid
    centroids_data.append({
        'Name': feature['properties']['Name'],
        # Include any other properties you want from the feature, e.g., 'BLK_NO'
        'Latitude': centroid.y,
        'Longitude': centroid.x
    })

# Convert the list of centroid data into a pandas DataFrame
df_centroids = pd.DataFrame(centroids_data)

# Return the pandas DataFrame
df_centroids

,Name,Latitude,Longitude
0,kml_1,1.449146,103.801493
1,kml_2,1.358925,103.750397
2,kml_3,1.353354,103.958286
3,kml_4,1.394533,103.749964
4,kml_5,1.279011,103.817563
...,...,...,...
12842,kml_12843,1.331494,103.927819
12843,kml_12844,1.358834,103.959075
12844,kml_12845,1.343808,103.689764
12845,kml_12846,1.393923,103.749375


In [4]:
import requests
import time
import pandas as pd
from shapely.geometry import shape
import json

from dotenv import load_dotenv
import os
import requests

load_dotenv()
payload = {
        "email": os.getenv("ONE_MAP_EMAIL"),
        "password": os.getenv("ONE_MAP_PASSWORD")
      }
api_key = requests.request("POST", "https://www.onemap.gov.sg/api/auth/post/getToken", json=payload)
api_key = api_key.json()["access_token"]

base_url = "https://www.onemap.gov.sg/api/public/revgeocode"

# Headers for the API request
headers = {
    "Authorization": f"Bearer {api_key}"
}

# Add a new column for addresses
df_centroids['Address'] = None

# Loop over the DataFrame rows
for index, row in df_centroids.iterrows():
    # Construct the API URL for reverse geocoding
    url = f"{base_url}?location={row['Latitude']},{row['Longitude']}&addressType=All&otherFeatures=N"

    # Make the API request
    response = requests.get(url, headers=headers)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the response JSON and get the address information
        data = response.json()
        if data.get('GeocodeInfo'):  # Check if there is geocode information in the response
            # Update the DataFrame with the address
            df_centroids.at[index, 'Address'] = data['GeocodeInfo'][0]['BUILDINGNAME']
        print(f"Reverse geocoded {row['Latitude']},{row['Longitude']} out of {len(df_centroids)}")
    else:
        print(f"Failed to reverse geocode. Status code: {response.status_code}, Response content: {response.text}")

    # Respect the API rate limits; add a slight delay between requests
    time.sleep(1)  # Adjust the delay as per the API provider's rate limit guidelines

# Save the DataFrame with the addresses to a CSV file
df_centroids.to_csv('../data/hdb_blocks_with_addresses.csv', index=False)
